In [1]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [2]:
# 28. dy
# 年频率。年终总股息除以市值。
df1 = pd.read_csv('../../data/csmar/季_相对价值指标.csv', usecols=['证券代码', '会计期间', '市值A'])
df1['年'] = df1['会计期间'].apply(lambda x: True if x[5:7] == '12' else False)
df1 = df1[df1['年']].reset_index()[['证券代码', '会计期间', '市值A']]
df2 = pd.read_csv('../../data/csmar/日_分配文件.csv', usecols=['证券代码', '除权(息)日期', '分配类型', '分配比率'])
df2 = df2[df2['分配类型'] == 'CA']
df2['会计期间'] = df2['除权(息)日期'].apply(lambda x: x[:5] + '12-31')
df3 = df2[['证券代码', '会计期间', '分配比率']].groupby(['证券代码', '会计期间']).sum().reset_index()
df4 = pd.read_csv('../../data/csmar/日_股本变动文件.csv', usecols=['证券代码', '股本变动日期', '总股数'])
df4 = df4.sort_values(by=['证券代码', '股本变动日期'])
df4['会计期间'] = df4['股本变动日期'].apply(lambda x: x[:5] + '12-31')
df5 = df4.drop_duplicates(subset=['证券代码', '会计期间'], keep='last')[['证券代码', '会计期间', '总股数']]
df6 = pd.merge(df1, df3, on=['证券代码', '会计期间'], how='left')
df6['分配比率'] = df6['分配比率'].fillna(0)
df7 = pd.merge(df6, df5, on=['证券代码', '会计期间'], how='left')
df7['总股数'] = df7['总股数'].fillna(axis=0, method='ffill')
df7['总股息'] = df7['总股数']*df7['分配比率']
df7['dy'] = df7['总股息']/df7['市值A'].replace(0, np.nan)

# 45. ms
# 年频率。根据Mohanram(2005)[4]的定义，基本绩效的八个指标变量的和。如下图：
df8 = df7[['证券代码', '会计期间', '总股息', 'dy']]
df9 = pd.read_csv('../../data/csmar/季_资产负债表.csv', usecols=['证券代码', '会计期间', '资产总计', '所有者权益合计'], low_memory=False)
df10 = pd.read_csv('../../data/csmar/季_利润表.csv', usecols=['证券代码', '会计期间', '净利润', '研发费用', '销售费用', '查询成功'])
df10['研发费用'][df10['查询成功'].astype('bool')] = df10['研发费用'][df10['查询成功'].astype('bool')].fillna(0)
df11 = pd.read_csv('../../data/csmar/季_现金流量表(直接法).csv', usecols=['证券代码', '会计期间', '经营活动产生的现金流量净额', '购建固定资产、无形资产和其他长期资产支付的现金'])
df12 = pd.merge(df8, df9, on=['证券代码', '会计期间'], how='left')
df13 = pd.merge(df12, df10, on=['证券代码', '会计期间'], how='left')
df14 = pd.merge(df13, df11, on=['证券代码', '会计期间'], how='left')
df14['ROA'] = df14['净利润']/df14['资产总计'].replace(0, np.nan)
df14['CFROA'] = df14['经营活动产生的现金流量净额']/df14['资产总计'].replace(0, np.nan)
df14['VARROA'] = df14[['证券代码', 'ROA']].groupby('证券代码').rolling(window=4).var().reset_index()['ROA']
df14['资本回报率'] = df14['净利润']/df14['所有者权益合计'].replace(0, np.nan)
df14['留存比率'] = (df14['净利润'] - df14['总股息'])/df14['净利润'].replace(0, np.nan)
df14['SGR'] = df14['资本回报率']*df14['留存比率']/(1 - df14['资本回报率']*df14['留存比率']).replace(0, np.nan)
df14['VARSGR'] = df14[['证券代码', 'SGR']].groupby('证券代码').rolling(window=4).var().reset_index()['SGR']
df14['RDINT'] = df14['研发费用']/df14['资产总计'].replace(0, np.nan)
df14['CAPINT'] = df14['购建固定资产、无形资产和其他长期资产支付的现金']/df14['资产总计'].replace(0, np.nan)
df14['ADINT'] = df14['销售费用']/df14['资产总计'].replace(0, np.nan)
df15 = pd.read_csv('../../data/csmar/公司文件.csv', usecols=['证券代码', '行业代码C'])
df16 = pd.merge(df14, df15, on=['证券代码'])
df17 = df16[['行业代码C', '会计期间', 'ROA', 'CFROA', 'VARROA', 'VARSGR', 'RDINT', 'CAPINT', 'ADINT']].groupby(['行业代码C', '会计期间']).median().reset_index()
df17.columns = ['行业代码C', '会计期间', 'ROA_indmed', 'CFROA_indmed', 'VARROA_indmed', 'VARSGR_indmed', 'RDINT_indmed', 'CAPINT_indmed', 'ADINT_indmed']
df18 = pd.merge(df16, df17, on=['行业代码C', '会计期间'], how='left')
df18['G1'] = np.where(df18['ROA'] >= df18['ROA_indmed'], 1, 0)
df18['G1'][df18['ROA'].isna()] = np.nan
df18['G2'] = np.where(df18['CFROA'] >= df18['CFROA_indmed'], 1, 0)
df18['G2'][df18['CFROA'].isna()] = np.nan
df18['G3'] = np.where(df18['CFROA'] >= df18['ROA'], 1, 0)
df18['G3'][df18['CFROA'].isna()] = np.nan
df18['G3'][df18['ROA'].isna()] = np.nan
df18['G4'] = np.where(df18['VARROA'] <= df18['VARROA_indmed'], 1, 0)
df18['G4'][df18['VARROA'].isna()] = np.nan
df18['G5'] = np.where(df18['VARSGR'] <= df18['VARSGR_indmed'], 1, 0)
df18['G5'][df18['VARSGR'].isna()] = np.nan
df18['G6'] = np.where(df18['RDINT'] >= df18['RDINT_indmed'], 1, 0)
df18['G6'][df18['RDINT'].isna()] = np.nan
df18['G7'] = np.where(df18['CAPINT'] >= df18['CAPINT_indmed'], 1, 0)
df18['G7'][df18['CAPINT'].isna()] = np.nan
df18['G8'] = np.where(df18['ADINT'] >= df18['ADINT_indmed'], 1, 0)
df18['G8'][df18['ADINT'].isna()] = np.nan
df18['ms'] = df18['G1'] + df18['G2'] + df18['G3'] + df18['G4'] + df18['G5'] + df18['G6'] + df18['G7'] + df18['G8']

df19 = df18[['证券代码', '会计期间', 'dy', 'ms']]
df19.to_csv('../../output/csmar/年_28_45.csv', index=False, encoding='utf-8-sig')
df19

,证券代码,会计期间,dy,ms
0,1,1990-12-31,NaN,NaN
1,1,1991-12-31,0.015971,NaN
2,1,1992-12-31,0.007758,NaN
3,1,1993-12-31,0.006769,NaN
4,1,1994-12-31,0.012345,NaN
...,...,...,...,...
180835,900957,2018-12-31,0.000000,NaN
180836,900957,2019-12-31,0.000000,NaN
180837,900957,2020-12-31,0.000000,NaN
180838,900957,2021-12-31,0.000000,NaN
